# Cross validation via MLJ

In [1]:
using MLJ
import RDatasets: dataset
import DataFrames: DataFrame, select
auto = dataset("ISLR", "Auto")
y, X = unpack(auto, ==(:MPG), col->true)
train, test = partition(eachindex(y), 0.5, shuffle=true, rng=444);

[ Info: Precompiling MLJ [add582a8-e3ab-11e8-2d5e-e98b27df1bc7]
[ Info: Precompiling RDatasets [ce6b1742-4840-55fa-b093-852dadbb1d8b]


In [3]:
LR = @load LinearRegressor pkg=MLJLinearModels

import MLJLinearModels ✔

[ Info: For silent loading, specify `verbosity=0`. 


MLJLinearModels.LinearRegressor

In [4]:
lm = LR()
mlm = machine(lm, select(X, :Horsepower), y) #Only use horsepower as predictor
fit!(mlm, rows=train)
rms(MLJ.predict(mlm, rows=test), y[test])^2

[ Info: Training Machine{LinearRegressor,…} @717.
┌ Info: Solver: MLJLinearModels.Analytical
│   iterative: Bool false
└   max_inner: Int64 200


23.493990895007986

Create 3 models up third order poly by creating a df with each order:

In [5]:
hp = X.Horsepower
Xhp = DataFrame(hp1=hp, hp2=hp.^2, hp3=hp.^3);

Create a pipeline, for selecting the desired feature. Accessed like a struct.